In [4]:
from Crypto.Util.number import getPrime, isPrime
import random


In [5]:
def generate_weak_prime(bitlength, n):
    order = n
    q_bits = bitlength - order.bit_length()
    while True:
        q = getPrime(q_bits)
        p = order * q + 1
        if isPrime(p):
            print(f"weak prime p: {p}")
            return p, order
    

def find_generator_order(g, p):
    value = 1
    for n in range(1, p):
        value = (value * g) % p
        if value == 1:
            print(f"generator order: {n}")
            return n
    return None


def generate_weak_generator(p, order):
    while True:
        i = random.randint(2, p - 2)
        g = pow(i, (p - 1) // order, p)
        if g != 1:
            print(f"weak generator g: {g}")
            return g

In [6]:
#diffie-hellman key exchange attack
p, order = generate_weak_prime(2048, 16)
g = generate_weak_generator(p, order)

a = random.getrandbits(256)
b = random.getrandbits(256)

#public keys
a_public_key = pow(g, a, p)
b_public_key = pow(g, b, p)

#shared secret key
a_secret_key = pow(b_public_key, a, p)
b_secret_key = pow(a_public_key, b, p)

#brute force attack
n = find_generator_order(g, p)
table = {pow(g, x, p): x for x in range(n)}
a_mod = table[a_public_key]
b_mod = table[b_public_key]
print(f"attacker found a mod {n}: {a_mod}")
print(f"attacker found b mod {n}: {b_mod}")

#calculate shared secret key
attacker_key = pow(g, (a_mod * b_mod) % n, p)
print(f"attacker computed shared secret key: {attacker_key}")
if attacker_key == a_secret_key:
    print("attacker successfully computed the shared secret key")
else:
    print("attacker failed to compute the shared secret key")

weak prime p: 14525444386954653839697566543725836967176383555687413669820656307798321844365920697682151712506564092811824122198127350451820815524582740183288839546617499178891983120197794584337510044102078272664995982271111270875228500536942152657275604671128512434248293562609834337831699508089710082933110948134605310286226541240403521399300648659839770000437917436917962145191181975881840161385619844251999126464570775531247635412757775433633256260608577685119343361878360032037056915076744801866174113214147395121122838470447448319072520431421808014829610218973762254173662394847233115354520504332766940653873904456890667851857
weak generator g: 12290759474934077128537716252251228368056418270147384680474387845867098678557114693849022901599738287444207271177297931697997275623951634142980844411118105151102355571251183285303903186567870642674989638163773662934954645408226539963874671922841873601983467765633384681915260639017222960840975498640076899610352623490617612335614778997396143028030488